# [SemEval 2023 Task 1](https://raganato.github.io/vwsd/)

[By Abdullah Alshadadi (Srking501)](https://github.com/Srking501)

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import keras
import random
import os

2023-03-05 17:35:11.929878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 17:35:12.466076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-05 17:35:12.466130: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-05 17:35:12.466136: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# to access google drive folder
# from google.colab import drive
# drive.mount('/content/drive') # When you run this you'll be prompted for a token - follow the link to generate this.
# #        "/content/drive/MyDrive/...path/to/dir
# path = "/content/drive/MyDrive/semeval-2023/task-1/"
# path = "/content/drive/MyDrive/data/"

# Comment this out if you are using Google Colab, otherwise just change to local directory where the data is located
path = "./data/"

In [3]:
os.getcwd()

'/home/srking501/Desktop/Projects/semeval-2023-task1'

❗❗❗Retrieve the data from [SemEval-2023 Task-1 page](https://raganato.github.io/vwsd/), specifically the ["[TRAIN+TRIAL]" data](https://drive.google.com/file/d/1byX4wpe1UjyCVyYrT04sW17NnycKAK7N/view), and put it into `data/` directory❗❗❗

In [4]:
from zipfile import ZipFile

try:
    if os.path.isdir(path + "semeval-2023-task-1-V-WSD-train-v1") == False:
        with ZipFile(path + "semeval-2023-task-1-V-WSD-train-v1.zip", "r") as zip:
            print("Unzipping data...")
            zip.extractall("data/")
            print("Finished.")
    else:
        print("[Data already unzipped]")
except FileNotFoundError:
    print(f"The file semeval-2023-task-1-V-WSD-train-v1.zip is not found in path:\n"
          f"{path}\n")
    print(f"The contents of {path}:\n"
          f"{os.listdir(path)}")

[Data already unzipped]


To check if we are in the right directory

In [5]:
os.listdir(path)

['semeval-2023-task-1-V-WSD-train-v1',
 'captions_train_clean_complete.csv',
 'README.md',
 'wordnet(individual tokens).csv']

Using `train_v1/` data

In [6]:
path_to_data = path + "semeval-2023-task-1-V-WSD-train-v1/"
data_labels = pd.read_csv(path_to_data + "train_v1/train.data.v1.txt", 
                sep = "\t", 
                names = ["target_word", "full_phrase", "image_1", "image_2", "image_3", "image_4",
                         "image_5", "image_6", "image_7", "image_8", "image_9", "image_10"])

In [7]:
data_labels.head()

,target_word,full_phrase,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,image_10
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg


In [8]:
gold_labels = pd.read_csv(path_to_data + "train_v1/train.gold.v1.txt", 
                    names = ["gold_images"])

In [9]:
gold_labels.head()

,gold_images
0,image.0.jpg
1,image.20.jpg
2,image.35.jpg
3,image.55.jpg
4,image.75.jpg


Image Caption Data and WordNet senses

In [10]:
caption_data_labels = pd.read_csv(path + "captions_train_clean_complete.csv")
wordnet_senses_labels = pd.read_csv(path + "wordnet(individual tokens).csv")

In [11]:
caption_data_labels.head()

,File Name,Generated Caption
0,image.0.jpg,a black and white bird is standing in the air
1,image.1.jpg,a man with a red hat is standing in front of a...
2,image.10.jpg,a man is standing in a large white and white a...
3,image.100.jpg,a man and a woman are playing with a toy
4,image.1000.jpg,a person is riding a dirt path in a wooded area


In [12]:
wordnet_senses_labels.head()

,Complex_word,senses
0,moorhen swamphen,black gallinule that inhabits ponds and lakes ...
1,serinus genus,Old World finches; e.g. canaries and serins or...
2,pegmatite igneous,produced under conditions involving intense he...
3,bangalore torpedo,an industrial city in south central India (wes...
4,bonxie skua,gull-like jaeger of northern seas or gull-like...


## Creating the main `dataset`

In [13]:
dataset = data_labels.join(gold_labels)

In [14]:
dataset.head()

,target_word,full_phrase,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,image_10,gold_images
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg,image.55.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg


The only important dataframes are `caption_data_labels_renamed` and the newly merged `wordnet_senses_labels_renamed` into the main `dataset` dataframe

In [17]:
# renaming columns to match the `data_labels` dataframe and 
# to fit one standard format 
# (following the python naming convention - 
# https://peps.python.org/pep-0008/#naming-conventions)
#
caption_data_labels_renamed = caption_data_labels.rename(columns={
    "File Name": "images",
    "Generated Caption": "generated_caption"
    })
wordnet_senses_labels_renamed = wordnet_senses_labels.rename(columns={
    "Complex_word": "full_phrase"
})

In [18]:
caption_data_labels_renamed.head()

,images,generated_caption
0,image.0.jpg,a black and white bird is standing in the air
1,image.1.jpg,a man with a red hat is standing in front of a...
2,image.10.jpg,a man is standing in a large white and white a...
3,image.100.jpg,a man and a woman are playing with a toy
4,image.1000.jpg,a person is riding a dirt path in a wooded area


In [19]:
wordnet_senses_labels_renamed.head()

,full_phrase,senses
0,moorhen swamphen,black gallinule that inhabits ponds and lakes ...
1,serinus genus,Old World finches; e.g. canaries and serins or...
2,pegmatite igneous,produced under conditions involving intense he...
3,bangalore torpedo,an industrial city in south central India (wes...
4,bonxie skua,gull-like jaeger of northern seas or gull-like...


In [20]:
dataset = dataset.merge(wordnet_senses_labels_renamed, how="inner", on="full_phrase")

In [21]:
dataset.head()

,target_word,full_phrase,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,image_10,gold_images,senses
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg,black gallinule that inhabits ponds and lakes ...
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg,Old World finches; e.g. canaries and serins or...
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg,produced under conditions involving intense he...
3,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg,gull-like jaeger of northern seas or gull-like...
4,ixia,ixia genus,image.90.jpg,image.3.jpg,image.91.jpg,image.4.jpg,image.92.jpg,image.1.jpg,image.2.jpg,image.94.jpg,image.93.jpg,image.5.jpg,image.90.jpg,a monocotyledonous genus of the family Iridace...
